In [0]:
import requests
from IPython.core.display import HTML
HTML(f"""
<style>
@import "https://cdn.jsdelivr.net/npm/bulma@0.9.4/css/bulma.min.css";
</style>
""")

# Gradient image features: Histogram of Oriented Gradients (HoG) introduction
This exercise is centered around extraction of gradient features from images. It consist of two parts:
1. Construction of sobel (derivative) filters and convolving them with a sample image.
2. Introduction and application of HoG features on a sample image.

The following cell loads a sample image.


In [0]:
## load input image
import matplotlib.pyplot as plt


fn =  "./data/people01.jpg"
image = plt.imread(fn)
print(image.shape)

fig, ax1 = plt.subplots(figsize=(12, 8), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

<article class="message task"><a class="anchor" id="gradient_features"></a>
    <div class="message-header">
        <span>Task 1: Apply sobel filters</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-stoplights medium"></i>
        </span>
    </div>
<div class="message-body">


The goal in this task is to construct derivative (sobel) filters in the x and y direction, and apply them by implementing a convolution algoritm.
The following steps must be completed: 
1. Constuct the two gradient filters defined by the matrices below:


$$
sobel_x=\left[
	\begin{array}{r r r}
		-1 & 0 & 1 \\
		-2 & 0 & 2 \\
		-1 & 0 & 1
	\end{array}
\right],
sobel_y=\left[
	\begin{array}{r r r}
		-1 & -2 & -1 \\
		0 &  0 & 0 \\
		1 & 2 & 1
	\end{array}
\right]
$$
2. (Optional) For the exercises last week, the `convolve`
 method from scipy was used to apply the filters. You are free to reuse the same method for this exercise. Optionally, this week we implement the method `convolve2d`
 from scratch. Below is a breakdown of the `convolve2d`
 function used for performing a 2D convolution operation on an image:
    - Reverse the filter, flipping it both vertically and horizontally, to adhere to the convolution's mathematical definition.

    - Output preparation, create an empty array, the same size as the image, to capture the convolution results.

    - Add zero-padding around the image borders to ensure that the kernel properly processes the edges.

    - Convolution process, iterate over each pixel. For each, apply the kernel, multiply its values with the image segment, and sum the results to assign a new value to the pixel.



3. Apply the sobel filters using `convolve2d`
 (or `convolve`
 from scipy) on the sample image. Plot the results using the provided subplot template.




</div></article>



In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve

def rgb2gray(rgb):
    """Convert RGB image to grayscale
       Parameters:
        rgb : RGB image
       Returns:
        Grayscale image
    """
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def convolve2d(image, kernel):
    """Perform 2D convolution on an image using a given kernel
       Parameters:
        image : 2D array
        kernel : 2D kernel array
       Returns:
        Convolved 2D array
    """
    write implementation here...


# Convert the image to grayscale
gray_image = rgb2gray(image)

# Compute the gradients using the Sobel operator
# Define the Sobel operator kernels.

# Step 1: construct sobel filters 
...
# Step 2: implement / finish
...
# Step 3: convolve the image with the filters
...
# Step 4:  Visualize the original image and gradients with the following subplot template

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(gray_image, cmap='gray'),
plt.title('Original Grayscale Image')
plt.xticks([]), plt.yticks([])  # Hides the tick values on X and Y axis

plt.subplot(1, 3, 2)
plt.imshow(<place_gradient_x_image>, cmap='gray')
plt.title('Gradient along X-axis')
plt.xticks([]), plt.yticks([])  # Hides the tick values on X and Y axis

plt.subplot(1, 3, 3)
plt.imshow(<place_gradient_y_image>, cmap='gray')
plt.title('Gradient along Y-axis')
plt.xticks([]), plt.yticks([])  # Hides the tick values on X and Y axis
plt.show()

In the following section, you are reintroduced (see the classification lecture slides) to the [HOG descriptor](https://en.wikipedia.org/wiki/Histogram_of_oriented_gradients)
, shown how to calculate a HoG featues for an image, which includes a visualization of the HoG descriptor.
## Overview of Histogram of Oriented Gradients (HOG) Algorithm
The Histogram of Oriented Gradients (HOG) is a feature descriptor used in computer vision and image processing to detect objects. The process involves the following stages:
**1. Preprocessing / Smoothing:**


Initially, the image undergoes a global normalization application — a procedure that diminishes the effects of lighting discrepancies. A common technique involves gamma (power-law) compression, which can be achieved by calculating the square root or logarithm of each color channel. This stage helps in lessening the impact of local shadowing and highlights variations since the image texture strength is generally related to the local surface illumination.
**2. Calculation of Image Gradients:**


This stage focuses on computing the first-order image gradients. These gradients are crucial as they encapsulate information regarding contours, silhouettes, and textures. This process helps in maintaining resistance to different lighting conditions. The standard procedure is to either convert the image to grayscale or use the most prominent color channel. <article class="message is-info">
  <div class="message-header">Info</div>
  <div class="message-body">

  This step was implemented by us in the first part of the exercise.


  </div>
</article>
 
**3. Creation of Cells and Calculation of Histograms:**


The objective here is to create an encoding that is attuned to local image content, yet stable against minor alterations in posture or appearance. This method involves partitioning the image window into smaller spatial segments known as "cells." Each cell undergoes a process to accumulate a local 1-D histogram that calculates gradient or edge orientations across all pixels within the cell. This histogram, known as the "orientation histogram," categorizes the gradient angle range into several predetermined bins, with the gradient magnitudes of the cell's pixels contributing to the histogram's data.
**4. Normalization Across Blocks:**


The subsequent stage involves normalizing, which takes into account local arrays of cells, and performs contrast adjustments on their collective responses before the next phase. This step is pivotal for achieving invariance to lighting, shadowing, and edge contrast. It operates by gathering a metric of local histogram "energy" over localized cell groups termed "blocks." This metric is then employed to normalize each cell within the block. Notably, while each cell might be common to multiple blocks, the normalizations are unique to each block, resulting in the cell's repeated appearance with distinct normalizations in the final output vector. Despite seeming redundant, this technique enhances performance. The descriptors obtained from these normalised blocks are what constitute the Histogram of Oriented Gradient (HOG) descriptors.
**5. Compilation of Feature Vector:**


The concluding step amalgamates the HOG descriptors from all block groupings, derived from a dense, overlapping grid spanning the detection window, into a unified feature vector. This vector is integral for the functionality of the window classifier.
In essence, the HOG algorithm is a sophisticated method that converts image data into a format conducive to object detection, with considerable resistance to variations in lighting and posture.
<article class="message task"><a class="anchor" id="descriptor"></a>
    <div class="message-header">
        <span>Task 2: Calculate the HOG descriptor</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-stoplights easy"></i>
        </span>
    </div>
<div class="message-body">


After block normalization, concatenate the resulting histograms into the final HOG feature descriptor (vector). 
Use the function [skimage.feature.hog](https://scikit-image.org/docs/stable/api/skimage.feature.html<elem-0>.feature.hog)
 to compute 
the feature descriptor vector from the resized image. Use the command:
```python
(fd, hog) = hog(
image,
orientations=bins,
pixels_per_cell=(pixels_per_cell, pixels_per_cell),
cells_per_block=(cells_per_block, cells_per_block),
block_norm="L2",
visualize=True,
feature_vector=False,
channel_axis=-1)


```
The function `skimage.features.hog`
 also returns the feature descriptor vector (`fd`
), in which its
size is equal to the number of: Bins $\times$ Block Columns
$\times$ Block Rows $\times$ Cells in the Block.


</div></article>



In [0]:
from skimage.feature import hog
from skimage import data, exposure

# load new image


image1 = data.astronaut()[:270,100:320]
print(image1.shape)

feature_vector=True,
bins = 8 
pixels_per_cell = 16
cells_per_block = 4 

fd1,hog_feat1 = image # replace this with hog feature results. fd is the feature vector and hog_image is the visualization of the calculated gradients.

# after implementation of the HoG function, run the following plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image1, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_feat1, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

<article class="message task"><a class="anchor" id="test"></a>
    <div class="message-header">
        <span>Task 3: Test the HOG descriptor</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-stoplights easy"></i>
        </span>
    </div>
<div class="message-body">


Test the HOG descriptor and function settings: 
Try changing:
- The number of bins
- The number of pixels per cell
- The number of cells per block

1. The number of cells per block does not change the visulization, why is that? what does it actually affect?
2. How much does the features space reduce by doing HoG (compare to the default setting)?    - Follow-up (optional), how is the number of output features (i.e. shape of HoG feature vector) defined by the settings (number of bins, number of pixels per cell, and number of cells per block) <article class="message is-warning">
  <div class="message-header">Tip</div>
  <div class="message-body">

  Use the output shape of the feature vector to infer the relationship.


  </div>
</article>






</div></article>

In the next cell we load another image (of a cat) and run the same code as above


In [0]:
image2 = data.cat()[:270,:440:2]
print(image2.shape)

fd2,hog_image2 = image # replace this with hog feature results. fd is the feature vector and hog_image is the visualization of the calculated gradients.

# after implementation of the HoG function, run the following plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image2, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled2 = exposure.rescale_intensity(hog_feat2, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled2, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')

<article class="message task"><a class="anchor" id="compare_images"></a>
    <div class="message-header">
        <span>Task 4: Use Hog Features</span>
        <span class="has-text-right">
          <i class="bi bi-code"></i><i class="bi bi-stoplights easy"></i>
        </span>
    </div>
<div class="message-body">


In this task compare the HoG features extracted from the two images. Specifically do the following:
1. For different setting visually compare the hog features
2. Calculate the normalized (use `np.linalg.norm`
 to normalize vectors) scalar product between the two feature vectors `fd1`
 and `fd2`
.     - Was is the scalar product between two feacture vectors? (compare it to the value of 1 )
    - Repeat the calculation for multiple HoG filter settings, notice a difference? 



#tip(Set `feature_vector`
=True in `hog`
)
3. Based on the similarity, explain how HoG features can be used to classify different images of cat/humans.



</div></article>



In [0]:
calculate the similarity here..

<article class="message task"><a class="anchor" id="interpretation"></a>
    <div class="message-header">
        <span>Task 5: Interpretation/ Understanding questions</span>
        <span class="has-text-right">
          <i class="bi bi-lightbulb-fill"></i><i class="bi bi-stoplights medium"></i>
        </span>
    </div>
<div class="message-body">


Test your implementation by selecting
multiple images available in our dataset (`peopleXY.jpg`
). You
have to compare (visually) the HOG features of people and non-people
images. 
- What is visually needed for humans to identify objects visually, compared to hog features?
- How would you apply HoG features in practice to classify different objects in a larger image with multiple people? 



</div></article>

We will get back to that next week!
The code in the cell below visualizes the HoG features:


In [0]:
# load new image
fn =  "./data/people01.jpg"
image = plt.imread(fn)
print(image.shape)

feature_vector=True,
bins = 8 
pixels_per_cell = 16
cells_per_block = 4 

fd,hog_image = image # replace this hog feature restults. fd is the feature vector and hog_image the visualization of the calculated gradients.

print(f' HoG feature vector shape: {fd.shape}')
print(f' HoG feature vector shape after vectorization: {fd.reshape(-1).shape}')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_feat, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')

plt.show()